<a href="https://colab.research.google.com/github/Jaeeyun/cv_final_assn/blob/main/cv_final_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 필요한 라이브러리 설치 및 업그레이드
!pip install --upgrade transformers accelerate
!pip install pandas scikit-learn torch torchvision opencv-python Pillow

# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 73.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.2
    Uninstalling transformers-4.57.2:
      Successfully uninstalled transformers-4.57.2
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import os
from pathlib import Path
from sklearn.preprocessing import LabelEncoder

def create_dataframe_and_encode_labels(base_dir):
    base_dir = Path(base_dir)
    image_paths = []
    labels = []

    print(f"'{base_dir}' 디렉토리에서 데이터 로딩 중...")

    for class_dir in sorted(base_dir.iterdir()):
        if class_dir.is_dir() and not class_dir.name.startswith('.'):
            label_name = class_dir.name # 예: 'Chorionic_villi'
            for image_file in class_dir.glob('*'):
                if image_file.suffix.lower() in ['.png', '.jpg', '.jpeg', '.tif', '.tiff']:
                    image_paths.append(str(image_file.resolve()))
                    labels.append(label_name)

    df = pd.DataFrame({'image_path': image_paths, 'label_name': labels})

    le = LabelEncoder()
    df['label'] = le.fit_transform(df['label_name'])

    print(f"\n총 {len(df)}개의 이미지 경로와 라벨을 매핑했습니다.")
    print(f"클래스 매핑: {list(le.classes_)}")
    print("최종 숫자 라벨 분포:\n", df['label'].value_counts().sort_index())

    return df


In [4]:
import pandas as pd
import os
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
import torch
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import AutoImageProcessor, AutoModelForImageClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import gc

torch.cuda.empty_cache()

# 설정
TRAIN_DATA_DIR = '/content/drive/MyDrive/cv_final/POC_Dataset/Training'
MODEL_NAME = "owkin/phikon-v2" # 사용할 Phikon 모델 ID
NUM_CLASSES = 4 # 데이터셋의 클래스 수 (0, 1, 2, 3)


# 1. DataFrame 로드 함수 (이전과 동일)
def create_dataframe_and_encode_labels(base_dir):
    base_dir = Path(base_dir)
    image_paths = []
    labels = []
    print(f"'{base_dir}' 디렉토리에서 데이터 로딩 중...")
    for class_dir in sorted(base_dir.iterdir()):
        if class_dir.is_dir() and not class_dir.name.startswith('.'):
            label_name = class_dir.name
            for image_file in class_dir.glob('*'):
                if image_file.suffix.lower() in ['.png', '.jpg', '.jpeg', '.tif', '.tiff']:
                    image_paths.append(str(image_file.resolve()))
                    labels.append(label_name)
    df = pd.DataFrame({'image_path': image_paths, 'label_name': labels})
    le = LabelEncoder()
    df['label'] = le.fit_transform(df['label_name'])
    print(f"\n총 {len(df)}개의 이미지 경로와 라벨을 매핑했습니다.")
    print(f"클래스 매핑: {list(le.classes_)}")
    return df

# 2. DataFrame 로드 및 분할
df = create_dataframe_and_encode_labels(base_dir=TRAIN_DATA_DIR)
train_df, val_df = train_test_split(df, test_size=0.3, stratify=df['label'], random_state=42)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

# 3. Phikon 프로세서 및 모델 로드
processor = AutoImageProcessor.from_pretrained(MODEL_NAME)
model = AutoModelForImageClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_CLASSES,
    ignore_mismatched_sizes=True
)

# 4. 사용자 정의 Dataset 클래스
class PhikonDataset(Dataset):
    def __init__(self, dataframe, processor):
        self.dataframe = dataframe
        self.processor = processor
    def __len__(self):
        return len(self.dataframe)
    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['image_path']
        label = self.dataframe.iloc[idx]['label']
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        inputs = self.processor(images=image, return_tensors="pt")
        pixel_values = inputs['pixel_values'].squeeze(0)
        return {'pixel_values': pixel_values, 'labels': torch.tensor(label, dtype=torch.long)}

train_dataset = PhikonDataset(train_df, processor)
val_dataset = PhikonDataset(val_df, processor)

# 5. 평가 지표 계산 함수 정의
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # 다중 클래스 분류 average='weighted' 사용
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1_weighted': f1,
        'precision_weighted': precision,
        'recall_weighted': recall,
    }

# 6. 학습 인자 설정
training_args = TrainingArguments(
    output_dir="./phikon_results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model="f1_weighted",
    report_to="none",
    fp16=True,
)

#7. Trainer 인스턴스 생성 및 학습 시작
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

print("\n--- Phikon 모델 파인튜닝 시작 ---")
trainer.train()
print("--- 학습 완료 ---")

# 최종 평가 지표 출력
results = trainer.evaluate()
print("\n--- 최종 검증 결과 (Precision, Recall, F1-Score 포함) ---")
print(results)


'/content/drive/MyDrive/cv_final/POC_Dataset/Training' 디렉토리에서 데이터 로딩 중...

총 4167개의 이미지 경로와 라벨을 매핑했습니다.
클래스 매핑: ['Chorionic_villi', 'Decidual_tissue', 'Hemorrhage', 'Trophoblastic_tissue']


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/750 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json: 0.00B [00:00, ?B/s]

...

Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at owkin/phikon-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



--- Phikon 모델 파인튜닝 시작 ---


Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted,Precision Weighted,Recall Weighted
1,No log,0.183628,0.936051,0.934931,0.938233,0.936051
2,No log,0.155169,0.940847,0.939888,0.944583,0.940847
3,0.237000,0.141751,0.932854,0.932143,0.933541,0.932854


--- 학습 완료 ---



--- 최종 검증 결과 (Precision, Recall, F1-Score 포함) ---
{'eval_loss': 0.15516948699951172, 'eval_accuracy': 0.9408473221422862, 'eval_f1_weighted': 0.93988825828169, 'eval_precision_weighted': 0.9445830749557879, 'eval_recall_weighted': 0.9408473221422862, 'eval_runtime': 18.6384, 'eval_samples_per_second': 67.119, 'eval_steps_per_second': 4.239, 'epoch': 3.0}


In [5]:
FINAL_SAVE_PATH = '/content/drive/MyDrive/cv_final'

trainer.save_model(FINAL_SAVE_PATH)
processor.save_pretrained(FINAL_SAVE_PATH)

print(f"\n✅ 최종 학습된 베스트 모델이 '{FINAL_SAVE_PATH}'에 성공적으로 저장되었습니다.")


✅ 최종 학습된 베스트 모델이 '/content/drive/MyDrive/cv_final'에 성공적으로 저장되었습니다.


In [10]:
import pandas as pd
from sklearn.metrics import classification_report


TEST_DATA_DIR = '/content/drive/MyDrive/cv_final/POC_Dataset/Testing'

# 1. 테스트 데이터셋 DataFrame 생성
print(f"\n--- '{TEST_DATA_DIR}' 디렉토리에서 테스트 데이터 로딩 중 ---")
# 테스트 데이터 로드 시 학습 데이터셋과 동일한 클래스 순서 및 인코딩을 가정
# 학습 시 사용한 LabelEncoder 인스턴스 사용
test_df = create_dataframe_and_encode_labels(base_dir=TEST_DATA_DIR)

# 2. 테스트 데이터셋 인스턴스 생성
test_dataset = PhikonDataset(test_df, processor)

# 3. trainer.predict() 메서드를 사용하여 예측 수행
print("\n--- 테스트 데이터셋에 대한 예측 수행 중 ---")
predictions = trainer.predict(test_dataset)

# 예측 결과 가져오기
preds = predictions.predictions.argmax(-1)
labels = predictions.label_ids

# 4. 결과 DataFrame에 추가 및 확인
test_df['predicted_label'] = preds
# 숫자 라벨을 원래 클래스 이름으로 다시 매핑 (학습 데이터의 클래스 매핑 순서를 따름)
# train_df에서 클래스 순서를 유추합니다.
label_map = dict(zip(df['label'], df['label_name']))
test_df['predicted_label_name'] = test_df['predicted_label'].map(label_map)


print("\n--- 예측 결과 확인 (샘플 5개) ---")
print(test_df[['image_path', 'label_name', 'predicted_label_name']].head())


# 5. 성능 평가
if labels is not None and len(labels) == len(preds):
    print("\n--- 전체 테스트 데이터셋 성능 리포트 ---")

    # 총 예측 개수 계산
    total_count = len(labels)

    # 맞은 개수 계산 (preds와 labels이 일치하는 경우의 합)
    correct_count = (preds == labels).sum()

    # 정답 개수/총 이미지 개수 계산
    accuracy = accuracy_score(labels, preds)

    print(f"총 이미지 개수: {total_count}개")
    print(f"정답 개수: {correct_count}개")
    print(f"정답 개수/총 이미지 개수: {accuracy:.4f}\n") # 소수점 4자리까지 표시

    # 정밀도, 재현율, F1-스코어 계산
    print(classification_report(labels, preds, target_names=list(df['label_name'].unique())))
else:
    print("\n테스트 데이터셋에 실제 라벨이 없거나 길이가 맞지 않아 성능 리포트를 생성할 수 없습니다.")


--- '/content/drive/MyDrive/cv_final/POC_Dataset/Testing' 디렉토리에서 테스트 데이터 로딩 중 ---
'/content/drive/MyDrive/cv_final/POC_Dataset/Testing' 디렉토리에서 데이터 로딩 중...

총 1511개의 이미지 경로와 라벨을 매핑했습니다.
클래스 매핑: ['Chorionic_villi', 'Decidual_tissue', 'Hemorrhage', 'Trophoblastic_tissue']

--- 테스트 데이터셋에 대한 예측 수행 중 ---



--- 예측 결과 확인 (샘플 5개) ---
                                          image_path       label_name  \
0  /content/drive/MyDrive/cv_final/POC_Dataset/Te...  Chorionic_villi   
1  /content/drive/MyDrive/cv_final/POC_Dataset/Te...  Chorionic_villi   
2  /content/drive/MyDrive/cv_final/POC_Dataset/Te...  Chorionic_villi   
3  /content/drive/MyDrive/cv_final/POC_Dataset/Te...  Chorionic_villi   
4  /content/drive/MyDrive/cv_final/POC_Dataset/Te...  Chorionic_villi   

  predicted_label_name  
0      Chorionic_villi  
1      Chorionic_villi  
2      Chorionic_villi  
3      Chorionic_villi  
4      Chorionic_villi  

--- 전체 테스트 데이터셋 성능 리포트 ---
총 이미지 개수: 1511개
정답 개수: 1287개
정답 개수/총 이미지 개수: 0.8518

                      precision    recall  f1-score   support

     Chorionic_villi       0.94      0.98      0.96       390
     Decidual_tissue       0.75      0.67      0.71       349
          Hemorrhage       0.78      1.00      0.88       421
Trophoblastic_tissue       1.00      0.71      0.83    

In [11]:
import torch
import pandas as pd
import numpy as np
import cv2
from pathlib import Path
from torch.utils.data import DataLoader
from transformers import AutoImageProcessor, AutoModelForImageClassification
from sklearn.metrics import classification_report, accuracy_score
from torch.utils.data import Dataset # PhikonDataset 클래스를 위해 임포트

SAVED_MODEL_PATH = '/content/drive/MyDrive/cv_final'
TEST_DATA_DIR = '/content/drive/MyDrive/cv_final/POC_Dataset/Testing'


# PhikonDataset 클래스는 이전 코드와 동일하며 상단에 미리 실행하여 정의했다고 가정

# 1. 저장된 모델 및 프로세서 로드
print(f"\n--- '{SAVED_MODEL_PATH}' 에서 모델 로드 중 ---")
# AutoModelForImageClassification.from_pretrained()를 사용하여 저장된 경로에서 직접 모델 가중치와 구성을 불러오기
model = AutoModelForImageClassification.from_pretrained(SAVED_MODEL_PATH)
processor = AutoImageProcessor.from_pretrained(SAVED_MODEL_PATH)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# 2. 테스트 데이터셋 로드 및 준비
# create_dataframe_and_encode_labels 함수는 사용자의 원본 학습 코드 상단에 정의되었다고 가정
print(f"\n--- '{TEST_DATA_DIR}' 디렉토리에서 테스트 데이터 로딩 중 ---")
test_df = create_dataframe_and_encode_labels(base_dir=TEST_DATA_DIR)
# 학습 시 사용한 df 변수 (전체 데이터셋)에서 클래스 이름을 가져와 일관된 매핑을 수행
class_names = list(df['label_name'].unique())

test_dataset = PhikonDataset(test_df, processor)

# 3. PyTorch DataLoader 설정
BATCH_SIZE = 16
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# 4. 직접 예측 루프 실행
print("\n--- 저장된 모델로 직접 예측 수행 중 ---")
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        pixel_values = batch['pixel_values'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(pixel_values)
        predictions = outputs.logits.argmax(dim=-1)

        all_preds.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 5. 결과 분석: 맞은 개수 및 정확도 계산
test_df['predicted_label'] = all_preds

total_count = len(all_labels)
correct_count = (np.array(all_preds) == np.array(all_labels)).sum()
accuracy = accuracy_score(all_labels, all_preds)

print("\n--- 테스트 예측 결과 요약 ---")
print(f"총 이미지 개수: {total_count}개")
print(f"정답 개수 (맞춘 개수): {correct_count}개")
print(f"정확도 (Accuracy): {accuracy:.4f}")

# 6. 상세 성능 리포트
print("\n--- 전체 테스트 데이터셋 성능 리포트 (저장된 모델 사용) ---")
print(classification_report(all_labels, all_preds, target_names=class_names))


--- '/content/drive/MyDrive/cv_final' 에서 모델 로드 중 ---

--- '/content/drive/MyDrive/cv_final/POC_Dataset/Testing' 디렉토리에서 테스트 데이터 로딩 중 ---
'/content/drive/MyDrive/cv_final/POC_Dataset/Testing' 디렉토리에서 데이터 로딩 중...

총 1511개의 이미지 경로와 라벨을 매핑했습니다.
클래스 매핑: ['Chorionic_villi', 'Decidual_tissue', 'Hemorrhage', 'Trophoblastic_tissue']

--- 저장된 모델로 직접 예측 수행 중 ---

--- 테스트 예측 결과 요약 ---
총 이미지 개수: 1511개
정답 개수 (맞춘 개수): 1287개
정확도 (Accuracy): 0.8518

--- 전체 테스트 데이터셋 성능 리포트 (저장된 모델 사용) ---
                      precision    recall  f1-score   support

     Chorionic_villi       0.94      0.98      0.96       390
     Decidual_tissue       0.75      0.67      0.71       349
          Hemorrhage       0.78      1.00      0.88       421
Trophoblastic_tissue       1.00      0.71      0.83       351

            accuracy                           0.85      1511
           macro avg       0.87      0.84      0.84      1511
        weighted avg       0.86      0.85      0.85      1511

